In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## import 用到的套件

In [2]:
import numpy as np
import time
import torch
import torch.nn.functional as F
from torchvision import transforms,models
import matplotlib.pyplot as plt
import matplotlib.font_manager as plt_font
folder="/content/drive/MyDrive/解密AI黑盒子分享/"
twfont1 = plt_font.FontProperties(fname=folder+"字型/kaiu.ttf")
import sys
sys.path.append(folder+'模組/')
import webcam

## 是否有GPU可以使用

In [3]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('device', device)

device cuda:0


## 測試資料預處理


In [4]:
test_transform=transforms.Compose([
    transforms.Resize(256),#改變大小
    transforms.CenterCrop(224),#中央裁剪
    transforms.ToTensor(),#轉成Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],std=[0.229, 0.224, 0.225])#像素資料歸一化
])

## 載入Resnet18預訓練模型

In [5]:
model=models.resnet18(weights='DEFAULT')  #使用訓練好的參數
model=model.to(device)          #是否使用GPU
model.eval()                #設定為測試模式
model                   #查看模型

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 載入ImageNet的1000種物品索引對應

In [6]:
import pandas as pd
df = pd.read_csv(folder+"資料集/ImageNet/index2class.csv")
index2label={}
for idx,row in df.iterrows():
  index2label[row['ID']]=row['Chinese']

In [7]:
print(index2label)

{0: '丁鯛', 1: '金魚', 2: '大白鯊', 3: '虎鯊', 4: '錘頭鯊', 5: '電鰩', 6: '黃貂魚', 7: '公雞', 8: '母雞', 9: '鴕鳥', 10: '燕雀', 11: '金翅雀', 12: '家朱雀', 13: '燈芯草雀', 14: '靛藍雀,靛藍鳥', 15: '藍鵐', 16: '夜鶯', 17: '松鴉', 18: '喜鵲', 19: '山雀', 20: '河鳥', 21: '鳶（猛禽）', 22: '禿頭鷹', 23: '禿鷲', 24: '大灰貓頭鷹', 25: '歐洲火蠑螈', 26: '普通蠑螈', 27: '水蜥', 28: '斑點蠑螈', 29: '蠑螈,泥狗', 30: '牛蛙', 31: '樹蛙', 32: '尾蛙,鈴蟾蜍,肋蟾蜍,尾蟾蜍', 33: '紅海龜', 34: '皮革龜', 35: '泥龜', 36: '淡水龜', 37: '箱龜', 38: '帶狀壁虎', 39: '普通鬣蜥', 40: '美國變色龍', 41: '鞭尾蜥蜴', 42: '飛龍科蜥蜴', 43: '褶邊蜥蜴', 44: '鱷魚蜥蜴', 45: '毒蜥', 46: '綠蜥蜴', 47: '非洲變色龍', 48: '科莫多蜥蜴', 49: '非洲鱷,尼羅河鱷魚', 50: '美國鱷魚,鱷魚', 51: '三角龍', 52: '雷蛇,蠕蟲蛇', 53: '環蛇,環頸蛇', 54: '希臘蛇', 55: '綠蛇,草蛇', 56: '國王蛇', 57: '襪帶蛇,草蛇', 58: '水蛇', 59: '藤蛇', 60: '夜蛇', 61: '大蟒蛇', 62: '岩石蟒蛇,岩蛇,蟒蛇', 63: '印度眼鏡蛇', 64: '綠曼巴', 65: '海蛇', 66: '角腹蛇', 67: '菱紋響尾蛇', 68: '角響尾蛇', 69: '三葉蟲', 70: '盲蜘蛛', 71: '蠍子', 72: '黑金花園蜘蛛', 73: '穀倉蜘蛛', 74: '花園蜘蛛', 75: '黑寡婦蜘蛛', 76: '狼蛛', 77: '狼蜘蛛,狩獵蜘蛛', 78: '壁虱', 79: '蜈蚣', 80: '黑松雞', 81: '松雞,雷鳥', 82: '披肩雞,披肩榛雞', 83: '草原雞,草原松雞', 84: '孔雀', 85: '鵪鶉'

## 即時物品影像辨識

In [8]:
webcam.video_stream()   #開始從webcam串流影像
count = 0
Starttime=time.time()
HTML=""         #影片上方要顯示的文字 
while True:
  img = webcam.video_frame(HTML)          #取得目前的影像frame，傳入對應的圖片說明
  if not img:
    break
  img = test_transform(img).unsqueeze(0).to(device) #目前的影像作預處裡
  output = model(img)                 #做前向傳播，得到各分類的分數
  softmax_out = F.softmax(output,dim=1).squeeze()   #對各分類做softmax運算，得到分類機率
  values,indexs = torch.topk(softmax_out,5)     #取可信度最大的5個结果：分類分數，分類索引
  Output=""
  for i in range(len(indexs)):
    Output +="{}({:.1f}%)".format(index2label[int(indexs[i])],values[i].item()*100)
  count+=1
  Stoptime=time.time()
  HTML =str(round(count/(Stoptime-Starttime),2))+"FPS<br>"+Output

<IPython.core.display.Javascript object>